In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
#optional 2023 spcific run hashed out

df = pd.read_csv('./gateio_data/aggregated_trade_data.csv')
#df = pd.read_csv('./gateio_data/aggregated_trade_data_23season.csv')
token_lookup = pd.read_csv('./football_strat/fanTokens.csv')
#matches_old = pd.read_csv("./football_strat/matchdays - Sheet1.csv")
btc_rets = pd.read_csv("./gateio_data//btc_daily_rets.csv")
#btc_rets = pd.read_csv("./gateio_data/btc_daily_rets_23season.csv")
matches = pd.read_csv("./football_strat/matchdays_new.csv")
#matches = pd.read_csv("./football_strat/matchdays_2023season.csv")

In [ ]:

#getting 'Club'/name variable into matches df
matches['Club'] = matches['tickers'].map(token_lookup[['ticker','Name']].set_index('ticker').to_dict()['Name'])

In [ ]:
df = df.sort_values(['ticker', 'timestamp'])

In [ ]:
token_lookup['ticker'] = [str(i)+"_USDT"  for i in token_lookup['ticker']]

In [ ]:
matches['Date'] = matches['matchdays']

In [ ]:
matches = matches[[len(i)>1 for i in matches['Date']]]
splits = [i.split(" ") for i in matches['Date']]
months = [datetime.strptime(i[1], '%b').month for i in splits]
days = [i[2].split(",")[0] for i in splits]
years = [i[3].split(",")[0] for i in splits]
matches['Day'] = days
matches['Month'] = months
matches['Year'] = years

In [ ]:
matches['DateTime'] = matches['Day'].astype(str)+"/"+matches['Month'].astype(str)+"/"+matches['Year'].astype(str)
matches['DateTime'] = pd.to_datetime(matches['DateTime'], format= "%d/%m/%Y")
matches['MatchDay'] = 1

In [ ]:
matches = matches.set_index('Club').join(token_lookup.set_index('Name'))

In [ ]:
btc_rets['timestamp'] = pd.to_datetime(btc_rets['timestamp'])
btc_rets.set_index('timestamp',inplace=True)
btc_rets.rename(columns={'vwap':'btc'}, inplace=True)
btc_rets['btc'] = btc_rets['btc'].rolling(2).sum()

In [ ]:
df['ticker'].drop_duplicates()

In [ ]:
agg_df = pd.DataFrame()
for each_club in df['ticker'].drop_duplicates():
    
    #filter to one club to avoid pct_change between clubs
    club_df = df.loc[df['ticker'] == each_club]
    club_df['timestamp'] = pd.to_datetime(club_df['timestamp'])
    club_df.set_index('timestamp', inplace=True)
    club_df = club_df['vwap'].pct_change().resample('d').sum()
    returns = pd.DataFrame(club_df.rolling(2).sum())
    returns['ticker'] = each_club

    #reset index so merge sets it
    returns.reset_index(inplace=True)
    merge_df = pd.merge(returns, matches,  how='left', left_on=['timestamp','ticker'], right_on = ['DateTime','ticker'])
    merge_df = merge_df.dropna(axis=0)
    merge_df = merge_df[['timestamp','vwap','ticker']]
    
    merge_df.set_index('timestamp', inplace=True)
    merge_df.drop_duplicates(inplace=True)
    
    merge_df = pd.concat([merge_df, btc_rets], axis=1)
    
    agg_df = pd.concat([agg_df, merge_df], axis=0)

In [ ]:
#+ becuase strategy returns are short returns
agg_df['strat_rets'] = agg_df['btc']-agg_df['vwap']

In [ ]:
agg_df = agg_df.dropna()

In [ ]:
agg_df.sort_index(inplace=True)

In [ ]:
agg_df['vwap'].cumsum().plot()

In [ ]:
agg_df[agg_df['ticker'] == 'CITY_USDT']['vwap'].head(40).cumsum().plot()
agg_df[agg_df['ticker'] == 'CITY_USDT']['strat_rets'].head(40).cumsum().plot()

In [ ]:
agg_df[agg_df['ticker'] == 'CITY_USDT']['vwap'].std()

In [ ]:
btc_rets.std()

In [ ]:
matches[matches['ticker'] == "CITY_USDT"].set_index("DateTime").sort_index().tail(170).head(66)

In [ ]:
agg_df[agg_df['ticker'] == "CITY_USDT"]['strat_rets'].tail(60)

In [ ]:
(agg_df['vwap']*500).cumsum().plot()

In [ ]:
agg_df[agg_df['vwap'].abs() > 0.05]['strat_rets'].cumsum().plot()

In [ ]:
(agg_df[agg_df['ticker']!='LAZIO_USDT']['vwap']*500).cumsum().plot()

In [ ]:
matches.index.drop_duplicates()

In [ ]:
agg_df.groupby('ticker').mean()

In [ ]:
agg_df['strat_rets'].mean()

In [ ]:
agg_df.groupby('ticker').std()['vwap']/np.sqrt(agg_df.groupby('ticker').count()['vwap'])

In [ ]:
(agg_df.groupby('ticker').mean()['vwap'])/(agg_df.groupby('ticker').std()['vwap']/np.sqrt(agg_df.groupby('ticker').count()['vwap']))

In [ ]:
matches[['DateTime', 'ticker']]

In [ ]:
agg_df